In [1]:
import pandas as pd # for dataframe manipulation
import numpy as np
import matplotlib.pyplot as plt # for plotting

from numpy import *

Load the data file for the 5-year Swedish death rates for the time period 1751 to 2017

In [26]:
# Insert the Total columns as a series
total = pd.read_csv('datasets/Swed_Mx_1x5.csv', sep=' ', header=0,
                   dtype={'Male':np.float64, 'Female':np.float64, 'Total':np.float64},
                   skip_blank_lines=True, infer_datetime_format=True, na_values='.')['Total']
# Convert the NaN's to 0 as they were initially supposed to be 0
# data.fillna(value=0)
total.head() # Reasonableness check

# write the cleaned data to a .txt file
total.to_csv('datasets/clean_data.csv', sep=',', encoding='utf-8')

In [35]:
# Convert the series into a matrix with 54 rows and 111 columns
# data = np.reshape(total, newshape=(54,111))
data = total.values.reshape(54,111)

In [61]:
# Transpose the matrix so that the ages are on the main axis and the years on the columns
data1 = data.transpose()
# Transform the resultant array into a Pandas dataframe
data_df = pd.DataFrame(data1)

# Drop all ages beyond age 90
data_df = data_df[0:90]
data_df.shape

(90, 54)

Estimate the value of a(x)

In [90]:
# Get the logarithm of the dataframe
data_log = np.log(data_df)

# Sum the logs across all the columns
data_sum = data_log.sum(axis = 1)

# Determine the estimated value of a, aHat
aHat = data_sum / len(data_df.columns)

# Singular Value Decomposition

Estimate the value of b(x) and k(t) by using Singular Value Decomposistion (SVD)

In [108]:
A_df = data_log - aHat

A_df.dropna(axis=1, inplace=True)

from scipy import linalg
u, d, vh = linalg.svd(A_df, full_matrices=True)
v = vh.transpose()

# To estimate parameters b(x) and k(t) , Lee-Carter Model use only rank r = 1
# estimation value of b(x) could be obtained as the result in the first column of matrix u
bHat = u[0]

# Meanwhile, the estimation value of k(t) could be achieved by the multiplication of first singular value and 
# first column of matrix V
kHat = d[0] * v[0]

From the estimate values above, mortality rate will be estimated by using Equation 1